In [26]:
# Import modules
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [27]:
# Load in data
train_df = pd.read_csv(Path('Resources/2019loans.csv'), index_col = 0)
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'), index_col = 0)

In [28]:
# Convert categorical data to numeric and separate target feature for training data
train_df = pd.get_dummies(train_df)
train_df.head()

,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
57107,57107,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,0.0,...,1,1,0,1,1,0,1,0,1,0
141451,141451,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,0.0,...,1,1,0,1,1,0,1,0,1,0
321143,321143,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,0.0,...,1,1,0,1,1,0,1,0,1,0
11778,11778,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,1.0,...,1,1,0,1,1,0,1,0,1,0
169382,169382,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,0.0,...,1,1,0,1,1,0,1,0,1,0


In [29]:
# Convert categorical data to numeric and separate target feature for testing data
test_df = pd.get_dummies(test_df)
test_df.head()

,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,loan_status_high_risk,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N
67991,67991,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,...,0,1,1,0,1,1,0,1,0,1
25429,25429,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,...,0,1,1,0,1,1,0,1,0,1
38496,38496,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,...,0,1,1,0,1,1,0,1,0,1
19667,19667,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,...,0,1,1,0,1,1,0,1,0,1
37505,37505,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,...,0,1,1,0,1,1,0,1,0,1


In [31]:
# add missing dummy variables to testing set
# Find missing column
missing_col = list(set(train_df.columns) - set(test_df.columns))
test_df[missing_col] = 0 * len(test_df)
test_df.head()

,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
67991,67991,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,...,1,1,0,1,1,0,1,0,1,0
25429,25429,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,...,1,1,0,1,1,0,1,0,1,0
38496,38496,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,...,1,1,0,1,1,0,1,0,1,0
19667,19667,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,...,1,1,0,1,1,0,1,0,1,0
37505,37505,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,...,1,1,0,1,1,0,1,0,1,0


In [46]:
# Train the Logistic Regression model on the unscaled data and print the model score

# Define x and y
X_train = train_df.loc[:, ~train_df.columns.isin(["index", 'loan_status_high_risk'])]
y_train = train_df['loan_status_high_risk']
X_test = test_df.loc[:, ~test_df.columns.isin(["index", 'loan_status_high_risk'])]
y_test = test_df['loan_status_high_risk']

# Define classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Print scores
print(f'Training Score: {clf.score(X_train, y_train)}')
print(f'Testing Score: {clf.score(X_test, y_test)}')

Training Score: 0.6516420361247948
Testing Score: 0.5161633347511697


/Users/nat/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [49]:
# Train a Random Forest Classifier model and print the model score
# Define classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Print scores
print(f'Training Score: {clf.score(X_train, y_train)}')
print(f'Testing Score: {clf.score(X_test, y_test)}')

Training Score: 1.0
Testing Score: 1.0


In [55]:
# Scale the data
scaler = StandardScaler().fit(train_df)
scaled_train_df = scaler.transform(train_df)
scaled_test_df = scaler.transform(test_df)
scaled_train_df = pd.DataFrame(scaled_train_df, columns = train_df.columns)
scaled_test_df = pd.DataFrame(scaled_test_df, columns = test_df.columns)

In [56]:
# Train the Logistic Regression model on the scaled data and print the model score
# Define x and y
X_scaled_train = scaled_train_df.loc[:, ~scaled_train_df.columns.isin(["index", 'loan_status_high_risk'])]
y_scaled_train = scaled_train_df['loan_status_high_risk']
X_scaled_test = scaled_test_df.loc[:, ~scaled_test_df.columns.isin(["index", 'loan_status_high_risk'])]
y_scaled_test = scaled_test_df['loan_status_high_risk']

# Define classifier
clf = LogisticRegression()
clf.fit(X_scaled_train, y_scaled_train)

# Print scores
print(f'Training Score: {clf.score(X_scaled_train, y_scaled_train)}')
print(f'Testing Score: {clf.score(X_scaled_test, y_scaled_test)}')

Training Score: 1.0
Testing Score: 0.9997873245427478


In [57]:
# Train a Random Forest Classifier model on the scaled data and print the model score
# Define classifier
clf = RandomForestClassifier()
clf.fit(X_scaled_train, y_scaled_train)

# Print scores
print(f'Training Score: {clf.score(X_scaled_train, y_scaled_train)}')
print(f'Testing Score: {clf.score(X_scaled_test, y_scaled_test)}')

Training Score: 1.0
Testing Score: 1.0
